### **Задание 1**   
Обучим нейронную сеть решать шифр Цезаря

*Что необходимо сделать:*

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

### **Задание 2**   
Выполнить практическую работу из лекционного ноутбука

1. Построить RNN-ячейку на основе полносвязных слоев
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

### Задание 1


#### 1. Написать алгоритм шифра Цезаря


> Шифр Цезаря - это простой тип подстановочного шифра, где каждая буква текста заменяется буквой с фиксированным числом позиций вниз по алфавиту.   
> Создадим слудующие переменные:  
> alphabets - словарь с алфавитом 
> K - для задания шага сдвига букв, которая будет вручную задаваться пользователем.   
> text - исходный текст
> encrypted_text - куда мы будем выводить зашифрованное сообщение.

In [38]:
# Создадим словарь
# Ключи 'ru' и 'en' используются для выбора соответствующего алфавита
alphabets = {
    'ru': 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя',
    'en': 'abcdefghijklmnopqrstuvwxyz'
}

In [41]:
# функция для шифра Цезаря
def caesar_cipher(text, K, alphabet):
    result = " "
    for char in text:
            if char.lower() in alphabet:
                is_upper = char.isupper()
                char = char.lower()
                index = (alphabet.find(char) + K ) % len(alphabet)
                shifted_char = alphabet[index].upper() if is_upper else alphabet[index]
                result += shifted_char
            else:
                result += char
    return result       

In [42]:
# Ввод данных
text = input('введите текст: ')
K = int(input('Шаг шифровки: '))
alphabet_choice = input('Выберите алфавит (ru/en): '). strip().lower()

# Выбор алфавита (по умолчанию русский)
alphabet = alphabets.get(alphabet_choice, alphabets['ru'])

# Шифрование и вывод результата
encrypted_text = caesar_cipher(text, K, alphabet)

print(f"""
Исходный текст: {text}
Шаг шифрования: {K}
Выбранный алфавит: {alphabet_choice}
Зашифрованный текст: {encrypted_text}
""")


Исходный текст: привет Мир
Шаг шифрования: 10
Выбранный алфавит: ru
Зашифрованный текст:  щътлоь Цтъ



#### 2. Создадим нейронную сеть   

Для решения задачи подойдет рекуррентная нейронная сеть (RNN) или её улучшенная версия — LSTM (Long Short-Term Memory). Эти архитектуры хорошо работают с последовательностями, такими как текст.

1. Подготовка данных

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [41]:
# Константы
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + list(CHARS)
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(INDEX_TO_CHAR)}

In [19]:
# Шифр Цезаря
def caesar_cipher(text, shift):
    return ''.join([INDEX_TO_CHAR[(CHAR_TO_INDEX.get(char, 0) - 1 + shift) % len(CHARS) + 1] 
                    if char in CHARS else char for char in text])

In [20]:
# Нейронная сеть
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(INDEX_TO_CHAR), 30)
        self.rnn = nn.RNN(30, 128, batch_first=True)
        self.out = nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        x = self.embedding(sentences)
        x, _ = self.rnn(x)
        return self.out(x)

In [21]:
# Генерация данных
def generate_dataset(texts, shifts):
    return [(caesar_cipher(text, random.choice(shifts)), text) for text in texts]

In [22]:
# Преобразование текста в тензор
def text_to_tensor(texts):
    max_len = max(len(text) for text in texts)
    tensor = torch.zeros((len(texts), max_len), dtype=torch.long)
    for i, text in enumerate(texts):
        for j, char in enumerate(text):
            tensor[i, j] = CHAR_TO_INDEX.get(char, 0)
    return tensor

In [23]:
# Дешифровка текста
def decrypt_text(model, encrypted_text):
    encrypted_tensor = text_to_tensor([encrypted_text])
    output = model(encrypted_tensor)
    _, indices = output.topk(1)
    return ''.join([INDEX_TO_CHAR[idx.item()] for idx in indices.flatten()])

In [36]:
# Обучение модели
def train_model(model, X, Y, epochs=50, lr=0.05):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        train_loss, train_accuracy = 0.0, 0.0
        for i in range(len(X)):
            optimizer.zero_grad()
            X_tensor = text_to_tensor([X[i]])
            Y_tensor = text_to_tensor([Y[i]])
            output = model(X_tensor).view(-1, len(INDEX_TO_CHAR))
            target = Y_tensor.view(-1)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(output, 1)
            train_accuracy += (predicted == target).sum().item() / len(target)
        
        print(f"Epoch {epoch}. Loss: {train_loss / len(X):.3f}, Accuracy: {train_accuracy / len(X):.3f}")

In [37]:
# Тестирование модели
def test_model(model, original_text, shift):
    encrypted_text = caesar_cipher(original_text, shift)
    decrypted_text = decrypt_text(model, encrypted_text)
    print(f"Original: {original_text}\nEncrypted: {encrypted_text}\nDecrypted: {decrypted_text}\n" + "-" * 40)

In [45]:
# Пример использования
texts = [
    "hello world", "neural network", "caesar cipher", "deep learning",
    "machine learning", "artificial intelligence", "data science",
    "python programming", "pytorch framework", "encryption algorithms"
]
shifts = [1, 2, 3, 4, 5]

In [46]:
X, Y = zip(*generate_dataset(texts, shifts))
model = Network()
train_model(model, X, Y)

Epoch 0. Loss: 3.333, Accuracy: 0.066
Epoch 1. Loss: 3.147, Accuracy: 0.229
Epoch 2. Loss: 2.961, Accuracy: 0.354
Epoch 3. Loss: 2.771, Accuracy: 0.411
Epoch 4. Loss: 2.583, Accuracy: 0.425
Epoch 5. Loss: 2.413, Accuracy: 0.407
Epoch 6. Loss: 2.267, Accuracy: 0.440
Epoch 7. Loss: 2.138, Accuracy: 0.460
Epoch 8. Loss: 2.019, Accuracy: 0.488
Epoch 9. Loss: 1.909, Accuracy: 0.511
Epoch 10. Loss: 1.807, Accuracy: 0.523
Epoch 11. Loss: 1.710, Accuracy: 0.541
Epoch 12. Loss: 1.620, Accuracy: 0.587
Epoch 13. Loss: 1.534, Accuracy: 0.608
Epoch 14. Loss: 1.453, Accuracy: 0.612
Epoch 15. Loss: 1.376, Accuracy: 0.641
Epoch 16. Loss: 1.302, Accuracy: 0.657
Epoch 17. Loss: 1.232, Accuracy: 0.676
Epoch 18. Loss: 1.165, Accuracy: 0.705
Epoch 19. Loss: 1.101, Accuracy: 0.734
Epoch 20. Loss: 1.040, Accuracy: 0.739
Epoch 21. Loss: 0.982, Accuracy: 0.739
Epoch 22. Loss: 0.926, Accuracy: 0.758
Epoch 23. Loss: 0.873, Accuracy: 0.768
Epoch 24. Loss: 0.822, Accuracy: 0.793
Epoch 25. Loss: 0.773, Accuracy: 0.

In [47]:
for text in texts:
    for shift in shifts:
        test_model(model, text, shift)

Original: hello world
Encrypted: nyiihebhviq
Decrypted: a glhminrka
----------------------------------------
Original: hello world
Encrypted: mkccnysnwcf
Decrypted: oella warli
----------------------------------------
Original: hello world
Encrypted: ptrrmkdmbra
Decrypted: hello world
----------------------------------------
Original: hello world
Encrypted: jzvvptqpsvx
Decrypted: p lcheah ia
----------------------------------------
Original: hello world
Encrypted:  gwwjzfjdwu
Decrypted: mtcrptlprra
----------------------------------------
Original: neural network
Encrypted: myovxiemyzbhvt
Decrypted: omacsiniewlorc
----------------------------------------
Original: neural network
Encrypted: pkhwucypkgsnwz
Decrypted: hencal oetchr 
----------------------------------------
Original: neural network
Encrypted: jtnborkjtldmbg
Decrypted: pearai peraore
----------------------------------------
Original: neural network
Encrypted:  zmshvt ziqpsl
Decrypted: m orarep lrnca
------------------------

In [48]:
# Шаг 1: Построение RNN-ячейки на основе полносвязных слоев
import torch
import torch.nn as nn

class CustomRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomRNNCell, self).__init__()
        self.hidden_size = hidden_size

        # Полносвязные слои
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)  # Вход + скрытое состояние -> скрытое состояние
        self.i2o = nn.Linear(input_size + hidden_size, output_size)  # Вход + скрытое состояние -> выход
        self.activation = nn.Tanh()  # Активация

    def forward(self, input, hidden):
        # Объединяем вход и скрытое состояние
        combined = torch.cat((input, hidden), dim=1)
        # Вычисляем новое скрытое состояние
        hidden = self.activation(self.i2h(combined))
        # Вычисляем выход
        output = self.i2o(combined)
        return output, hidden

    def init_hidden(self, batch_size):
        # Инициализация скрытого состояния
        return torch.zeros(batch_size, self.hidden_size)

In [51]:
# Шаг 2: Подготовка данных
# Загрузка данных (пример)
text = ('simpsons_script_lines')

# Создание словаря символов
chars = sorted(list(set(text)))
char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for char, idx in char_to_index.items()}

# Преобразование текста в индексы
text_as_int = [char_to_index[char] for char in text]

# Создание последовательностей для обучения
seq_length = 100  # Длина последовательности
examples = [text_as_int[i:i + seq_length + 1] for i in range(len(text_as_int) - seq_length)]

# Разделение на вход и цель
inputs = [example[:-1] for example in examples]
targets = [example[1:] for example in examples]

# Преобразование в тензоры
inputs = torch.tensor(inputs, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

In [52]:
# Шаг 3: Создание модели
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = CustomRNNCell(input_size, hidden_size, output_size)
        self.embedding = nn.Embedding(output_size, input_size)  # Эмбеддинг для символов

    def forward(self, input, hidden):
        # Применяем эмбеддинг
        embedded = self.embedding(input)
        # Проходим через RNN
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        return self.rnn.init_hidden(batch_size)

# Параметры модели
input_size = 128  # Размерность эмбеддинга
hidden_size = 256  # Размер скрытого состояния
output_size = len(chars)  # Размер словаря

# Создание модели
model = RNNModel(input_size, hidden_size, output_size)

In [55]:
# Шаг 4: Обучение модели
# Параметры обучения
batch_size = 64
epochs = 20
learning_rate = 0.001

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Обучение
for epoch in range(epochs):
    hidden = model.init_hidden(batch_size)
    total_loss = 0  # Для накопления потерь за эпоху

    for i in range(0, inputs.size(0), batch_size):
        # Получаем батч
        input_batch = inputs[i:i + batch_size]
        target_batch = targets[i:i + batch_size]

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        output, hidden = model(input_batch, hidden)
        loss = criterion(output.view(-1, output_size), target_batch.view(-1))

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        # Отсоединяем скрытое состояние для следующего батча
        hidden = hidden.detach()

        # Накопление потерь
        total_loss += loss.item()

    # Вывод средней потери за эпоху
    avg_loss = total_loss / (inputs.size(0) // batch_size)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss}")

ZeroDivisionError: division by zero

In [54]:
# Шаг 5: Генерация текста
def generate_text(model, start_string, num_generate=1000):
    # Преобразуем начальную строку в индексы
    input_eval = torch.tensor([[char_to_index[char] for char in start_string]], dtype=torch.long)

    # Генерация текста
    text_generated = []
    hidden = model.init_hidden(1)
    for _ in range(num_generate):
        output, hidden = model(input_eval, hidden)
        # Получаем следующий символ
        predicted_id = torch.argmax(output, dim=2)[-1].item()
        text_generated.append(index_to_char[predicted_id])
        # Обновляем вход
        input_eval = torch.tensor([[predicted_id]], dtype=torch.long)

    return start_string + ''.join(text_generated)

# Пример генерации текста
print(generate_text(model, start_string="Homer: "))

KeyError: 'H'